# Вебинар 1. Введение, примеры задач, бизнес- и ML-метрики 

![about_me.png](attachment:about_me.png)

-----

# 1. Введение и примеры задач

![recsys_everywhere.png](attachment:recsys_everywhere.png)

----

![skills.png](attachment:skills.png)

---

##  Постановка задачи

*Задача рекомендаций* - предложить пользователю товары, которые ему "понравятся"

Компоненты:
    - Пользователи (users)
    - Товары (items)
    - Взаимодействия (interactions) - по ним понимаем, что товар "понравился"

---

## Пользователи VS Товары

![ex_user_items.png](attachment:ex_user_items.png)


**Кейс**: определить в каждой ситуации, что является пользователем, а что - товаром
    
    1. На сайте Кинопоиска порекомендовать интересные фильмы
    2. На сайте hh.ru подобрать лучшие вакансии для соискателя вакансий
    3. На сайте hh.ru найти похожие вакансии на ту, которую сейчас просматривает пользователь ("вам также может понравиться")
    3. На сайте hh.ru подобрать лучших кандидатов для работодателя
    
*Пример ответа в чате:*
    1. Пользователь (П) - фильм, Товар (T) - фильм
    2. П - работодатель, Т - соискатель вакансий

---

## Взаимодействия --> "понравился товар"

![ex_interactions.png](attachment:ex_interactions.png)


---

## В чем отличие от классификации?

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.DataFrame([['Иван', 'Хлеб Бородинский', 1],
             ['Иван', 'Хлеб Белый', 0],
             ['Василий', 'Йогурт Epica', 1]], columns=['user', 'item', 'purchase_fact'])

,user,item,purchase_fact
0,Иван,Хлеб Бородинский,1
1,Иван,Хлеб Белый,0
2,Василий,Йогурт Epica,1


Постойте, ведь можно добавить фичи на user (средний чек, кол-во покупок в категориях и т.д), 
на item (цена, кол-во продаж в неделю и т.l), и решать задачу классификации. В чем отличие RecSys от классификации?

 - Много предсказаний для 1 user (extreme classification)
 - Гораздо больший объем данных: 100K users, 10K items --> 1B предсказаний
 - Большинство товаров user никогда не видел --> Не взаимодействовал --> 0 не значит "не понравился"
 - Нет явного таргета. Не понятно, что значит "(не) понравился"
 - Feedback loop
 - Иногдаь важен порядок рекомендаций

In [3]:
pd.DataFrame([['Иван', 'Хлеб Бородинский', 1],
              ['Иван', 'Хлеб Белый', 0],
              ['Иван', 'Йогурт Epica', "?"],
              ['Василий', 'Хлеб Бородинский', "?"],
              ['Василий', 'Хлеб Белый', "?"],
              ['Василий', 'Йогурт Epica', 1]], columns=['user', 'item', 'purchase_fact'])

,user,item,purchase_fact
0,Иван,Хлеб Бородинский,1
1,Иван,Хлеб Белый,0
2,Иван,Йогурт Epica,?
3,Василий,Хлеб Бородинский,?
4,Василий,Хлеб Белый,?
5,Василий,Йогурт Epica,1


----

## Основные типы задач:
    
**Рекомендация топ-К товаров**: Дан список товаров. Рекомендовать пользователю K товаров, которые ему понравятся
    - e-mail рассылки (М.Видео, Hoff, Пятерочка)
    - push-уведомления (Delivery Club, HeadHunter, Виктория)
    - Рекомендации в отдельном окне на сайте (vk, okko)

**Ранжирование товаров**: Дан список товаров. Нужно его отранжировать в порядке убывания интереса для пользователя
    - Ранжирование каталога товаров (Wildberries, Lamoda)
    - Ранжирование ленты (vk, Я.Дзен)
    - Ранжирование поисковой выдачи (Яндекс, Гугл)
    - Ранжирование "каруселей" товаров (Delivery Club, Я.Еда)
    
**Поиск похожих товаров**: Дан 1 товар. Нужно найти наиболее похожие на него товары
    - "Вам также может понравиться" (Ozon, Циан)
    - Похожим пользователям понравилось (vk)
    - Вы можете быть знакомы (vk, facebook)
    
**Рекомендация дополнителдьного товара**. Дан 1 товар. Найти товары, которые покупают вместе с этим товаром
    - С этим товаром часто покупают (Я.Маркет, Ozon, М.Видео)
    
   
---
На 7 вебинаре мы разберем рекомендательные системы из HeadHunter, Я.Дзен и vk

---

# 2. ML-mетрики качества

*Хороши ли рекомендации, предлагаемые моделью?*  
Обычно считаются для каждого юзера, затем усредняются по юзерам

Представим, что в магазине всего 10 товаров

In [4]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

### 1. Hit rate

Hit rate = был ли хотя бы 1 релевантный товар среди рекомендованных

- Иногда применяется, когда продаются достаточно дрогие товары (например, бытовая техника) 

----
Hit rate = (был ли хотя бы 1 релевантный товар среди рекомендованных)   

Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)

In [5]:
def hit_rate(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate


def hit_rate_at_k(recommended_list, bought_list, k=5):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])

    flags = np.isin(bought_list, recommended_list)

    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate

In [6]:
hit_rate(recommended_list, bought_list)

1

In [7]:
hit_rate_at_k(recommended_list, bought_list)

1

### 2. Precision

*Precision* - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров  юзер купил

- Пожалуй, самая приближенная к бизнес-метрикам и самая популярная метрика

---

Precision= (# of recommended items that are relevant) / (# of recommended items)  

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  

**Note:** Обычно k в precision@k достаточно невелико (5-20) и определяется из бизнес-логики. Например, 5 товаров в e-mail рассылке, 20 ответов на первой странице google и т.д

Красная рыба - 400 руб  
Молоко - 60 руб  
Хлеб = 40 руб  
Гречка = 40 руб  
Шоколад = 90 руб  

------  
Варенье - 240 руб  
...  

**Case 1**  
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [1, 0, 0, 0 , 1]  

$precison@5 = \frac{1 + 0 + 0 +0 + 1}{1+1+1+1+1} = 40\%$
$money precision@5 = \frac{1*400 + 0*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 77.7\%$  

  
**Case 2**   
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [0, 1, 0, 0 , 1]  

$precison@5 = \frac{0 + 1 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{0*400 + 1*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 15.8\%$

$precison@5 = \frac{1 + 0 + 0 +0 + 1}{1+1+1+1+1} = 40%$

In [8]:
prices_recommended = [400, 60, 40, 40 , 90]

In [9]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision


def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision


def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    bought_list = bought_list
    recommended_list = recommended_list[:k]

    flags = np.isin(recommended_list, bought_list)
    
    precision_at_k = (flags@prices_recommended) / sum(prices_recommended[:k])

    
    return precision_at_k

In [10]:
precision(recommended_list, bought_list)

0.2

In [11]:
precision_at_k(recommended_list, bought_list, k=5)

0.4

In [12]:
precision_at_k(recommended_list, bought_list, k=3)

0.3333333333333333

In [13]:
money_precision_at_k(recommended_list, bought_list, prices_recommended)

0.6984126984126984

### 3. Recall

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

- Обычно используется для моделей пре-фильтрации товаров (убрать те товары, которые точно не будем рекомендовать)

---

Recall= (# of recommended items that are relevant) / (# of relevant items)  

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)  

    
  
**Note:** в recall@k число k обычно достаточно большое (50-200), больше чем покупок у среднестатистического юзера

In [21]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(recommended_list, bought_list)
    
    recall = (flags@prices_recommended) / sum(prices_bought)
    
    return recall

In [22]:
recall(recommended_list, bought_list)

0.5

In [23]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought=[400, 60])

0.9565217391304348

---

# Метрики ранжирования

Если важен порядок рекомендаций. Подробнее можно почитать [здесь](https://habr.com/ru/company/econtenta/blog/303458/). Формулы в статье могут несколько отличаться от формул в лекции 

## AP@k
AP@k - average precision at k

$$AP\@k = \frac{1}{r} \sum{[recommended_{relevant_i}] * precision\@k}$$

- r - кол-во релевантный среди рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка реокмендаций

In [28]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(1, k+1):
        
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            sum_ += p_k
            
    result = sum_ / sum(flags)
    
    return result

In [29]:
ap_k(recommended_list, bought_list, k=5)

0.16666666666666666

### MAP@k

MAP@k (Mean Average Precision@k)
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций
$$MAP\@k = \frac{1}{|U|} \sum_u{AP_k}$$

|U| - кол-во юзеров

In [32]:
users = [f'user_{x}' for x in range(10)]
users

['user_0',
 'user_1',
 'user_2',
 'user_3',
 'user_4',
 'user_5',
 'user_6',
 'user_7',
 'user_8',
 'user_9']

In [34]:
def map_k(recommended_list, bought_list, k=5):

    result = ap_k(recommended_list, bought_list, k) / len(users)
    
    return result

In [35]:
map_k(recommended_list, bought_list)

0.016666666666666666

### AUC@k
AUC для первых k наблюдений  
- Можно посчитать как sklern.metrics.roc_auc_score для топ-k предсказаний
- Показывает долю верно отранжированных товаров

### NDCG@k
Normalized discounted cumulative gain

$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$  

$discount(i) = 1$ if $i <= 2$,   
$discount(i) = log_2(i)$ if $i > 2$


(!) Считаем для первых k рекоммендаций   
(!) - существуют вариации с другими $discount(i)$  
i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров 

$$NDCG = \frac{DCG}{ideal DCG}$$


In [36]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

$DCG\@5 = \frac{1}{5}*(1 / 1 + 0 / 2 + 0 / log(3) + 1 / log(4) + 0 / log(5))$

$ideal DCG\@5 = \frac{1}{5}*(1 / 1 + 1 / 2 + 1 / log(3) + 1 / log(4) + 1 / log(5))$

$NDCG = \frac{DCG}{ideal DCG}$

### MRR@k
Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$

$$MRR = mean(\frac{1}{k_u})$$

In [25]:
def reciprocal_rank(recommended_list, bought_list):
    # your_code
    return result

# 3. Связь бизнес-метрик, ML-метрик и функции потерь(loss)

- **loss** - то, что оптимизирует модель (RMSE - Root Mean Squared Error)
- **ML-метрика** - то, как мы измеряем качество на test (money precision@5)
- **Бизнес-метрика** - то, что хочет оптимизировать бизнес (выручка)

В идеале loss = ML-метрика = бизнес-метрика, но это возмонжо в очень редких случаях

Надеемся, что:  
    *Оптимизация loss --> рост ML-метрик --> рост бизнес-метрик*

### Пример: рекомендательная система товаров на сайте  
      
На сайте есть баннер, на котором мы можем разместить 5 товаров. Надо порекомендовать эти 5 товаров персонально каждому юзеру

*Шаг 1: Определим бизнес-метрику*  
Бизнес хочет максимизировать выручку --> бизнес-метрика - **Выручка**  

*Шаг 2: Разложим ее на составляющие*  
Выручка =   
       Средний чек * кол-во покупок =   
       Средний чек * (число юзеров * конверсия из захода на сайт в заказ) =
       Число юзеров * (Средний чек * конверсия из захода на сайт в заказ)
       
Рекомендательная система влияет только на (Средний чек * конверсия из захода на сайт в заказ).   
Хорошим приближением этого является **money precision@5** - ML-метрика

*Шаг 3: loss*  
Это существенно сложнее. Персонализированные ML-модели не умеют напрямую оптимизировать конверсию. Стандартный loss - **RMSE**. Можно для начала попробовать его. 

P.S. Если вы хотите приблизить RMSE к money precision@5, то можно посчитать weighted RMSE, где вес каждого наблюдения = стоимость товара.   
P.S.S. Про другие виды loss будет рассказано в курсе